# Sustainable retirement age
AIM: assuming that the current situation of working / retired people is sustainable, and keeping that as reference, what is the retirement age for the whole century, considering the forecast model of the population I have built in my previous Notebook? 

In [5]:
from istatapi import discovery, retrieval
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [6]:
dfpproj = (
    pd.read_csv("data/3_projection.csv", index_col=0)
    .rename(columns={ str(y): y for y in range(2002, 2100+1)})
)
dfpproj

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
age,,,,,,,,,,,,,,,,,,,,,
0,523007,535656,542323,558500,551127,556884,560700,570791,562322,553218,...,161619.148762,159232.450737,156900.326009,154621.723718,152400.672172,150230.228052,148110.659203,146040.429954,144022.287080,142054.465459
1,529233,526577,538020,545596,559067,552870,560028,563021,569963,561694,...,164067.774209,161619.148762,159232.450737,156900.326009,154621.723718,152400.672172,150230.228052,148110.659203,146040.429954,144022.287080
2,528131,533126,531043,540860,548487,560347,556146,562640,564317,569064,...,166587.956432,164067.774209,161619.148762,159232.450737,156900.326009,154621.723718,152400.672172,150230.228052,148110.659203,146040.429954
3,518790,530847,537507,535545,543768,550808,563000,559193,564536,565219,...,169178.688696,166587.956432,164067.774209,161619.148762,159232.450737,156900.326009,154621.723718,152400.672172,150230.228052,148110.659203
4,515957,522484,534064,542329,540131,545782,554754,565127,560899,565563,...,171844.386323,169178.688696,166587.956432,164067.774209,161619.148762,159232.450737,156900.326009,154621.723718,152400.672172,150230.228052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,16400,17254,17494,20055,21122,24346,25054,27290,28086,29780,...,134331.985311,134331.752241,136309.702742,137309.102341,139100.193025,139248.964015,141716.245814,139986.028883,139287.123833,141152.665733
97,10981,11797,11980,12758,14071,15356,17559,18179,20203,21305,...,109303.873325,108632.880640,108760.627003,110491.877383,111432.755480,113018.785510,113272.279707,115414.261491,114138.485959,113701.283586
98,6838,7643,7901,8505,8725,9888,10871,12349,12977,14740,...,88447.653690,85984.560871,85558.459993,85760.931356,87229.553511,88076.706383,89436.155392,89742.839829,91547.969983,90642.905474


In [7]:
# I'm making some assumptions on the average age of retirement and start working
working_age = { y: 20 + 2*(y-2002)/(2022-2002) for y in range(2002,2022+1) } # Assumption: it was 20 in 2002 and it is 22 in 2022
retirem_age = { y: 58 + 7*(y-2002)/(2022-2002) for y in range(2002,2022+1) } # Assumption: it was 58 in 2002 and it is 65 in 2022

# ratio of people in working age over retired age
wr_df = pd.DataFrame(index=range(2002,2022+1))
wr_df["working-age / retirement-age ratio"] = np.nan
for y in wr_df.index:
    wa = working_age[y]
    ra = retirem_age[y]
    wr_df.loc[y, "working-age / retirement-age ratio"] = dfpproj.loc[wa:ra,y].sum() / dfpproj.loc[ra:,y].sum()
    
wr_df

,working-age / retirement-age ratio
2002,2.021864
2003,2.048581
2004,2.052414
2005,2.177448
2006,2.165623
2007,2.113820
2008,2.246041
2009,2.207381
2010,2.163356
2011,2.290755


Note that I'm not consideting unemployment, assuming it is constant, and what matters is the ratio between working and retired people not if working people are actually employed or not.

I will now compute the projected retirement age that it is necessary to maintain an average working/retirement ratio of 2.2

In [8]:
wr_ratio = 2.2

# Same data plotted in two variants
raproj_df = pd.DataFrame(index=range(2002,2100+1))
raproj_df["Sustainable retirement age"] = np.nan
for y in raproj_df.index:
    wa = 22 # Assumption: people will start working at 22 on average
    for ra in range(50, 100): # I'm testing different retirement ages (ra) untill the ratio is 2.2 is reached
        if dfpproj.loc[wa:ra,y].sum() / dfpproj.loc[ra:,y].sum() > wr_ratio:
            raproj_df.loc[y, "Sustainable retirement age"] = ra
            break
    
fig = raproj_df.plot()
fig.update_layout(
    showlegend=False,
    xaxis_title="Year",
    yaxis_title="Retirement Age",
    title="Sustainable retirement age to maintain the same ratio of working/retired people of 2.2"
)
fig.show()

# Variant #2: year of birth on the x-axis
raproj_df["Born year retiring"] = raproj_df.index - raproj_df["Sustainable retirement age"]
fig = px.line(
    data_frame=raproj_df.reset_index().rename(columns={"index": "Year of Retirement"}),
    x="Born year retiring",
    y="Sustainable retirement age",
    hover_data=["Year of Retirement"],
) 
fig.update_layout(
    showlegend=False,
    xaxis_title="Year of Birth",
    yaxis_title="Retirement Age",
    title="Sustainable retirement age to maintain the same ratio of working/retired people of 2.2"
)
fig.show()

## Conclusions
- I made some assumptions on the age at which people starts to work and retired in the past
- I computed a ratio between working-age and retired-age people of 2.0 - 2.4 in the historical data, and picked an average value of 2.2 as the "referance" one to keep constant in the future
- Based on my projection of Italian future population, being born in 1990, I expect to retire at 76 year old in 2066

## Follow-up
- Consider unemployment: this is a self-balancing secondary effect as less woking people might compensate lower unemployment too
- Run a sensitivity test on the "reference" ratio of working-age/retirement-age people as it is a very handwavy estimate
- Get more data on effective historical start age and retirement age
- Run a sensitivity test on the age at which people will start to work
- Compute the life expectancy after retirement: is it fair w.r.t. those who retired in the past years?